In [89]:
import pandas as pd
import numpy as np
import matplotlib as plt
import re

In [125]:
stopwords_fr = []
with open("data/stopwords/stopwords-fr.txt", 'r', encoding='utf-8') as f:
    stopwords_fr = f.readlines()
    stopwords_fr = [w.strip() for w in stopwords_fr]
stopwords_fr[:10]

['a', 'à', 'abord', 'absolument', 'afin', 'ah', 'ai', 'aie', 'aient', 'aies']

In [145]:
punctuations_fr = "\"$%&'?!()*+,-./:;=@[\]^_`{|}'~«»´’–"

In [167]:
party_tweets = pd.read_csv("data/twitter_data/party_tweets.csv")
member_tweets = pd.read_csv("data/twitter_data/merged_member_tweets.csv")

In [201]:
party_tweets.head()

,id,timestamp,partyname,username,tweet_text,all_hashtags,all_mentions,all_urls,retweet_count,favorite_count,range,lang
0,1196812925580840961,Tue Nov 19 15:30:11 +0000 2019,UDC Suisse,UDCch,"Durant la session d’hiver à venir, le Parlemen...",[],[],['https://www.udc.ch/actualites/articles/commu...,0,5,259,fr
1,1196375024124538880,Mon Nov 18 10:30:07 +0000 2019,UDC Suisse,UDCch,"Le glissement à gauche, qui a eu lieu lors des...",[],[],['https://www.udc.ch/actualites/articles/commu...,1,9,218,fr
2,1189851774510874624,Thu Oct 31 10:29:04 +0000 2019,UDC Suisse,UDCch,"Au lieu d'attaquer le mal à la racine, le Cons...",[],[],['https://www.udc.ch/actualites/articles/commu...,2,7,145,fr
3,1187013387508748288,Wed Oct 23 14:30:19 +0000 2019,UDC Suisse,UDCch,Les conséquences nuisibles de l’immigration dé...,[],[],['https://www.udc.ch/actualites/articles/commu...,3,17,208,fr
4,1186266665358647298,Mon Oct 21 13:03:07 +0000 2019,UDC Suisse,UDCch,«Un tout grand Merci à toutes les électrices e...,[],[],[],7,28,272,fr


In [200]:
member_tweets.head()

,id,timestamp,member_name,party_name,username,tweet_text,all_hashtags,all_mentions,all_urls,retweet_count,favorite_count,range,lang
0,1196747623471996928,Tue Nov 19 11:10:42 +0000 2019,Christian Levrat,PS Suisse,ChristianLevrat,@patoudoux @nantermod Heureux pays... petits s...,[],"['patoudoux', 'nantermod']",[],0,5,56,fr
1,1196741312264970240,Tue Nov 19 10:45:37 +0000 2019,Christian Levrat,PS Suisse,ChristianLevrat,"RT @MikeMuellerLate: Liebe @claudia_blumer, ge...",[],"['MikeMuellerLate', 'claudia_blumer', 'tagesan...",[],27,0,140,de
2,1196064766248280066,Sun Nov 17 13:57:16 +0000 2019,Christian Levrat,PS Suisse,ChristianLevrat,"Des résultats convaincants, voire époustouflan...",[],"['MarinaCarobbio', 'zanettiroberto', 'PaulRech...",[],13,98,159,fr
3,1193533989333749760,Sun Nov 10 14:20:52 +0000 2019,Christian Levrat,PS Suisse,ChristianLevrat,Alors qu’on attend toujours les résultats (ent...,[],[],[],16,89,106,fr
4,1193083434790531073,Sat Nov 09 08:30:31 +0000 2019,Christian Levrat,PS Suisse,ChristianLevrat,"Merci d’y penser, de faire un effort et de pas...",[],[],[],7,41,194,fr


In [183]:
member_tweets['party_name'].unique()

array(['PS Suisse', 'CVP PDC PPD PCD', 'UDC Suisse',
       'Les VERTS suisses 🌻', "Vert'libéraux Suisse", 'PLR Suisse'],
      dtype=object)

In [180]:
party_tweets['partyname'].unique()

array(['UDC Suisse', 'PS Suisse', 'PLR Suisse', 'Les VERTS suisses 🌻',
       "Vert'libéraux Suisse", 'SVP Schweiz', 'SP Schweiz', 'FDP Schweiz',
       'GRÜNE Schweiz 🌻', 'Grünliberale Schweiz', 'CVP PDC PPD PCD'],
      dtype=object)

# Keywords of parties

In [188]:
from collections import Counter

def tweets_to_bag(tweets, lang, username):
    """
    lang: {'fr','de'}
    username: {'UDCch', 'PSSuisse', 'PLR_Suisse', 'LesVertsSuisses',
       'vertliberaux', 'SVPch', 'spschweiz', 'FDP_Liberalen', 'GrueneCH',
       'grunliberale', 'CVP_PDC'}
    
    """
    tweets_copy = tweets.copy()
    tweets_copy = tweets_copy[tweets_copy.lang==lang]
    tweets_copy = tweets_copy[tweets_copy.username==username]
    tweets_copy = tweets_copy["tweet_text"].str.lower()
    
    text = ""
    for t in tweets_copy:
        text += ' ' +  t
    text = re.sub(r'https:\/\/t\.co\/.{10}', '', text)
    
    if (lang=='fr'):
        for c in punctuations_fr:
            text = text.replace(c, ' ')
        for w in stopwords_fr:
            regex = r" " + w + " "
            text = re.sub(regex, ' ', text)
            
        text = re.sub(regex, ' ', text)
        text = text.replace('  ', ' ')
        text = text.replace('  ', ' ')
        text = text.replace('  ', ' ')
        
        return Counter(text.split(" ")).most_common()
    else:
        raise NotImplementedError # TODO

In [192]:
tweets_to_bag(party_tweets, lang='fr', username='UDCch')

[('udc', 275),
 ('suisse', 238),
 ('http', 184),
 ('co', 184),
 ('conseil', 150),
 ('fédéral', 137),
 ('ue', 96),
 ('initiative', 71),
 ('accord', 71),
 ('asile', 55),
 ('immigration', 54),
 ('pays', 50),
 ('politique', 48),
 ('cadre', 47),
 ('national', 41),
 ('délégués', 38),
 ('groupe', 36),
 ('gauche', 36),
 ('#libreetsûr', 36),
 ('institutionnel', 35),
 ('élections', 34),
 ('peuple', 34),
 ('veut', 33),
 ('sécurité', 33),
 ('loi', 32),
 ('#votezudc', 30),
 ('réforme', 30),
 ('démocratie', 30),
 ('ui', 29),
 ('droit', 29),
 ('assemblée', 28),
 ('faire', 26),
 ('étrangers', 26),
 ('libre', 25),
 ('commission', 25),
 ('pacte', 25),
 ('économie', 24),
 ('conseiller', 24),
 ('cantons', 24),
 ('lieu', 23),
 ('faut', 23),
 ('autodétermination', 23),
 ('projet', 22),
 ('🇨🇭', 22),
 ('suisses', 22),
 ('avs', 22),
 ('parti', 22),
 ('fédérales', 21),
 ('liberté', 21),
 ('campagne', 21),
 ('majorité', 21),
 ('vote', 21),
 ('parlement', 20),
 ('exige', 20),
 ('nu', 20),
 ('confédération', 19),


In [193]:
tweets_to_bag(party_tweets, lang='fr', username='PLR_Suisse')

[('plr', 1216),
 ('rt', 1199),
 ('suisse', 659),
 ('http', 541),
 ('co', 501),
 ('#chvote', 489),
 ('#ef2015', 418),
 ('#plr', 333),
 ('#ad', 256),
 ('conseil', 251),
 ('initiative', 200),
 ('#polch', 189),
 ('#parlch', 184),
 ('…', 158),
 ('rtsinfo', 150),
 ('vincentarlettaz', 143),
 ('petragoessi', 136),
 ('ui', 131),
 ('vd', 130),
 ('#avancerensemble', 129),
 ('#ef2019', 128),
 ('fédéral', 121),
 ('https', 113),
 ('national', 111),
 ('politique', 109),
 ('président', 101),
 ('nantermod', 98),
 ('débat', 97),
 ('conseiller', 94),
 ('br', 92),
 ('voter', 90),
 ('campagne', 89),
 ('lifrancais', 88),
 ('candidats', 87),
 ('jsa', 87),
 ('amp', 84),
 ('1', 82),
 ('voix', 81),
 ('loi', 79),
 ('hhiltpold', 76),
 ('délégués', 76),
 ('#chveut', 74),
 ('parti', 73),
 ('etats', 73),
 ('ctobre', 73),
 ('groupe', 72),
 ('votez', 72),
 ('journée', 72),
 ('#rendrepossible', 68),
 ('demain', 68),
 ('isabellemoret', 67),
 ('cf', 65),
 ('presse', 64),
 ('cc', 63),
 ('canton', 62),
 ('20', 62),
 ('pdc'

In [194]:
tweets_to_bag(party_tweets, lang='fr', username='PSSuisse')

[('http', 829),
 ('co', 817),
 ('rt', 597),
 ('ps', 443),
 ('suisse', 335),
 ('christianlevrat', 243),
 ('initiative', 233),
 ('conseil', 223),
 ('#adps', 214),
 ('ui', 196),
 ('pssuisse', 193),
 ('#chvote', 174),
 ('politique', 139),
 ('fédéral', 128),
 ('nordmannroger', 112),
 ('groupe', 94),
 ('mathiasreynard', 91),
 ('égalité', 91),
 ('socialiste', 90),
 ('réforme', 82),
 ('femmes', 79),
 ('alain', 75),
 ('berset', 75),
 ('jcschwaab', 70),
 ('udc', 70),
 ('majorité', 67),
 ('droits', 67),
 ('#pv2020', 67),
 ('national', 66),
 ('mesures', 66),
 ('plr', 65),
 ('ans', 65),
 ('projet', 62),
 ('avs', 62),
 ('travail', 61),
 ('primes', 60),
 ('#congresps', 60),
 ('#psad', 58),
 ('loi', 54),
 ('campagne', 50),
 ('…', 50),
 ('faveur', 49),
 ('sociale', 49),
 ('protection', 49),
 ('vote', 48),
 ('états', 47),
 ('parlement', 47),
 ('faire', 47),
 ('2', 47),
 ('10', 47),
 ('#sppsadv', 47),
 ('présidence', 46),
 ('service', 46),
 ('population', 45),
 ('salaires', 44),
 ('carlosommaruga', 44),


In [197]:
tweets_to_bag(party_tweets, lang='fr', username='LesVertsSuisses')

[('http', 785),
 ('co', 741),
 ('rt', 675),
 ('verts', 604),
 ('adelethorens', 326),
 ('initiative', 293),
 ('conseil', 263),
 ('suisse', 255),
 ('climat', 218),
 ('amp', 192),
 ('ui', 191),
 ('#vertsad', 182),
 ('fédéral', 148),
 ('lesvertssuisse', 135),
 ('protection', 126),
 ('politique', 126),
 ('#electionsclimat2019🌍', 118),
 ('parlement', 116),
 ('…', 105),
 ('regularytz', 102),
 ('national', 102),
 ('égalité', 97),
 ('loi', 96),
 ('parti', 91),
 ('2', 88),
 ('climatique', 86),
 ('droits', 85),
 ('voter', 83),
 ('#parlch', 82),
 ('vert', 81),
 ('environnement', 80),
 ('#chvote', 79),
 ('etats', 78),
 ('ge', 78),
 ('#adverts', 73),
 ('vertsvd', 72),
 ('verte', 69),
 ('#votezvert', 69),
 ('#vagueverte💚', 65),
 ('élections', 65),
 ('ctobre', 65),
 ('projet', 63),
 ('#alimentsequitables', 63),
 ('#ef2015', 62),
 ('bglaettli', 61),
 ('économie', 61),
 ('faut', 61),
 ('femmes', 59),
 ('groupe', 59),
 ('robertcramer', 58),
 ('berne', 58),
 ('1', 58),
 ('ch', 58),
 ('référendum', 57),
 (

In [198]:
tweets_to_bag(party_tweets, lang='fr', username='vertliberaux')

[('rt', 453),
 ('vert', 318),
 ('libéraux', 233),
 ('#créateursdavenir', 197),
 ('chevalley', 171),
 ('vertliberaux', 165),
 ('suisse', 164),
 ('#ef2019', 140),
 ('http', 132),
 ('co', 131),
 ('jvl', 116),
 ('initiative', 108),
 ('ch', 104),
 ('conseil', 86),
 ('vd', 72),
 ('ui', 71),
 ('libéral', 63),
 ('ge', 58),
 ('parti', 57),
 ('jeunes', 46),
 ('politique', 43),
 ('économie', 40),
 ('faire', 40),
 ('pvlvd', 38),
 ('juerg', 36),
 ('grossen', 36),
 ('accord', 36),
 ('fédéral', 35),
 ('votez', 34),
 ('loi', 34),
 ('kathrinbertschy', 34),
 ('national', 32),
 ('temps', 31),
 ('isfpo', 30),
 ('président', 30),
 ('climatique', 30),
 ('taxe', 30),
 ('campagne', 29),
 ('pays', 29),
 ('projet', 29),
 ('climat', 28),
 ('protection', 28),
 ('…', 28),
 ('#chvote', 28),
 ('libérale', 27),
 ('énergie', 27),
 ('transition', 26),
 ('voix', 26),
 ('grand', 25),
 ('environnement', 25),
 ('femmes', 25),
 ('berne', 24),
 ('délégués', 24),
 ('mwuarin', 24),
 ('écologique', 24),
 ('énergétique', 24),
 (

In [199]:
tweets_to_bag(party_tweets, lang='fr', username='CVP_PDC')

[('#pdc', 441),
 ('pdc', 339),
 ('rt', 240),
 ('initiative', 225),
 ('suisse', 199),
 ('conseil', 195),
 ('santé', 171),
 ('coûts', 166),
 ('➡️', 143),
 ('#chvote', 124),
 ('fédéral', 117),
 ('#adpdc', 107),
 ('#ef2019', 98),
 ('ui', 93),
 ('#suisse', 91),
 ('national', 87),
 ('conseiller', 84),
 ('politique', 81),
 ('frein', 79),
 ('gerhardpfister', 76),
 ('etats', 74),
 ('micgirod', 71),
 ('projet', 71),
 ('cvp', 58),
 ('#nobillag', 56),
 ('veut', 55),
 ('🇨🇭', 53),
 ('#pv2020', 52),
 ('primes', 50),
 ('#avs', 49),
 ('parti', 48),
 ('conseillère', 48),
 ('groupe', 47),
 ('loi', 44),
 ('#ue', 41),
 ('faire', 41),
 ('pays', 41),
 ('réforme', 41),
 ('voix', 40),
 ('parlch', 39),
 ('soutient', 38),
 ('pdcvr', 38),
 ('populaire', 38),
 ('fiscalité', 38),
 ('augmentation', 37),
 ('#rffa', 37),
 ('fédérale', 36),
 ('succès', 36),
 ('engagement', 35),
 ('pénalisation', 34),
 ('délégués', 34),
 ('système', 34),
 ('économie', 34),
 ('roduitbenjamin', 34),
 ('nouvelle', 34),
 ('radiotelesuisse',